In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT_NAME"] = os.getenv("LANGCHAIN_PROJECT_NAME")

In [9]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://medium.com/@kevinnokiawriting/my-ultimate-2025-blueprint-step-by-step-plan-for-achieving-success-695cc2b2bde9")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://medium.com/@kevinnokiawriting/my-ultimate-2025-blueprint-step-by-step-plan-for-achieving-success-695cc2b2bde9', 'title': 'My Ultimate 2025 Blueprint: Step-by-Step Plan for Achieving Success | by Kevin Nokia | Dec, 2024 | Medium', 'description': 'And it feels quick too, like I didn’t realize that I almost touched 2025 in 3 days (the day I write this article). My 2024 wasn’t my best, but I struggled a lot. I struggle with eliminating my bad…', 'language': 'en'}, page_content="My Ultimate 2025 Blueprint: Step-by-Step Plan for Achieving Success | by Kevin Nokia | Dec, 2024 | MediumOpen in appSign upSign inWriteSign upSign inMy Ultimate 2025 Blueprint: Step-by-Step Plan for Achieving SuccessSteal it, use it, adjust it, and implement it if you want.Kevin Nokia·Follow6 min read·Dec 30, 2024--19ListenSharePhoto by Kelly Sikkema on UnsplashI’ve learned a lot in 2024.And it feels quick too, like I didn’t realize that I almost touched 2025 in 3 days (the day

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)

In [12]:
splitted_docs = splitter.split_documents(docs)
splitted_docs

[Document(metadata={'source': 'https://medium.com/@kevinnokiawriting/my-ultimate-2025-blueprint-step-by-step-plan-for-achieving-success-695cc2b2bde9', 'title': 'My Ultimate 2025 Blueprint: Step-by-Step Plan for Achieving Success | by Kevin Nokia | Dec, 2024 | Medium', 'description': 'And it feels quick too, like I didn’t realize that I almost touched 2025 in 3 days (the day I write this article). My 2024 wasn’t my best, but I struggled a lot. I struggle with eliminating my bad…', 'language': 'en'}, page_content='My Ultimate 2025 Blueprint: Step-by-Step Plan for Achieving Success | by Kevin Nokia | Dec, 2024 | MediumOpen in appSign upSign inWriteSign upSign inMy Ultimate 2025 Blueprint: Step-by-Step Plan for Achieving SuccessSteal it, use it, adjust it, and implement it if you want.Kevin Nokia·Follow6 min read·Dec 30, 2024--19ListenSharePhoto by Kelly Sikkema on UnsplashI’ve learned a lot in 2024.And it feels quick too, like I didn’t realize that I almost touched 2025 in 3 days (the day

In [13]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
embedder = OpenAIEmbeddings()

In [14]:
db = FAISS.from_documents(splitted_docs, embedder)

In [15]:
db

In [16]:
query = "And it feels quick too, like I didn’t realize that I almost touched 2025 in 3 days (the day I write this article)."
retrieved_docs = db.similarity_search(query)

In [17]:
retrieved_docs

[Document(id='596dff5f-3ef3-483a-852a-70fa36a161c4', metadata={'source': 'https://medium.com/@kevinnokiawriting/my-ultimate-2025-blueprint-step-by-step-plan-for-achieving-success-695cc2b2bde9', 'title': 'My Ultimate 2025 Blueprint: Step-by-Step Plan for Achieving Success | by Kevin Nokia | Dec, 2024 | Medium', 'description': 'And it feels quick too, like I didn’t realize that I almost touched 2025 in 3 days (the day I write this article). My 2024 wasn’t my best, but I struggled a lot. I struggle with eliminating my bad…', 'language': 'en'}, page_content='My Ultimate 2025 Blueprint: Step-by-Step Plan for Achieving Success | by Kevin Nokia | Dec, 2024 | MediumOpen in appSign upSign inWriteSign upSign inMy Ultimate 2025 Blueprint: Step-by-Step Plan for Achieving SuccessSteal it, use it, adjust it, and implement it if you want.Kevin Nokia·Follow6 min read·Dec 30, 2024--19ListenSharePhoto by Kelly Sikkema on UnsplashI’ve learned a lot in 2024.And it feels quick too, like I didn’t realize th

In [18]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
Answerthe following question based on give context
<context>
{context}
</context>
"""
)

In [19]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [20]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)

In [22]:
retriever = db.as_retriever()
from langchain.chains import create_retrieval_chain
retrievel_chain = create_retrieval_chain(retriever,document_chain)

In [23]:
retrievel_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000245D768D250>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswerthe following question based on give context\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
            | ChatOpenAI(client=<

In [24]:
response = retrievel_chain.invoke({"input":"And it feels quick too, like I didn’t realize that I almost touched 2025 in 3 days (the day I write this article)."})

In [25]:
response

{'input': 'And it feels quick too, like I didn’t realize that I almost touched 2025 in 3 days (the day I write this article).',
 'context': [Document(id='596dff5f-3ef3-483a-852a-70fa36a161c4', metadata={'source': 'https://medium.com/@kevinnokiawriting/my-ultimate-2025-blueprint-step-by-step-plan-for-achieving-success-695cc2b2bde9', 'title': 'My Ultimate 2025 Blueprint: Step-by-Step Plan for Achieving Success | by Kevin Nokia | Dec, 2024 | Medium', 'description': 'And it feels quick too, like I didn’t realize that I almost touched 2025 in 3 days (the day I write this article). My 2024 wasn’t my best, but I struggled a lot. I struggle with eliminating my bad…', 'language': 'en'}, page_content='My Ultimate 2025 Blueprint: Step-by-Step Plan for Achieving Success | by Kevin Nokia | Dec, 2024 | MediumOpen in appSign upSign inWriteSign upSign inMy Ultimate 2025 Blueprint: Step-by-Step Plan for Achieving SuccessSteal it, use it, adjust it, and implement it if you want.Kevin Nokia·Follow6 min r

'Based on the provided context, the main lesson Kevin Nokia learned in 2024 that he plans to carry into 2025 is the importance of persistence and consistency. Despite facing difficulties like procrastination and bad habits, he emphasizes the significance of starting now, being consistent, and focusing on the process to achieve goals. He advises against waiting until a later time (like the start of a new year) to make progress and highlights the importance of tackling goals one at a time while minimizing distractions.'

In [27]:
type(response['answer'])

str